# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
# import xlsxwriter
import math
from scipy.stats import percentileofscore as score
from statistics import mean

## Importing Our List of Stocks

We'll need to import our list of stocks and our API token before proceeding. Please note that the list is a slightly modified version of the S&P 500. A few constituent stocks needed to be removed, as they were not publicly traded a year ago.

In [2]:
sp500_stocks = pd.read_csv('constituents_modified.csv')

from mysecrets2 import POLYGON_API_KEY

## Building our Database Through API Calls

We will create a new DataFrame called simple_df and use a for loop to populate the table with the following information: stock ticker symbol; current price of the stock; price of the stock one year ago; one-year return for the stock; and the number of shares to buy.

In [3]:
my_columns1 = ['Ticker', 'Price Today', 'Previous Price', 'One-Year Return', 'Number of Shares to Buy']
simple_df = pd.DataFrame(columns = my_columns1)

We will gather the required data from a service called Polygon (https://polygon.io). We will make API calls to their website and insert this data into our DataFrame. This process takes a while, as each stock requires two API calls.

In [4]:
cur_date = '2023-11-03'
prv_date = '2022-11-03'

for stk in sp500_stocks['Ticker'][:]:

    cur_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{cur_date}/{cur_date}?apiKey={POLYGON_API_KEY}'
    cur_data = requests.get(cur_api).json()
    cur_results0 = cur_data['results']
    cur_results1 = cur_results0[0]
    cur_price = cur_results1['c']

    prv_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{prv_date}/{prv_date}?apiKey={POLYGON_API_KEY}'
    prv_data = requests.get(prv_api).json()
    prv_results0 = prv_data['results']
    prv_results1 = prv_results0[0]
    prv_price = prv_results1['c']

    per_change = (cur_price - prv_price) / prv_price

    simple_df = simple_df.append(pd.Series([stk, cur_price, prv_price, per_change, 'N/A'], index = my_columns1), ignore_index = True)

/tmp/ipykernel_239/4081584381.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_df = simple_df.append(pd.Series([stk, cur_price, prv_price, per_change, 'N/A'], index = my_columns1), ignore_index = True)
/tmp/ipykernel_239/4081584381.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_df = simple_df.append(pd.Series([stk, cur_price, prv_price, per_change, 'N/A'], index = my_columns1), ignore_index = True)
/tmp/ipykernel_239/4081584381.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simple_df = simple_df.append(pd.Series([stk, cur_price, prv_price, per_change, 'N/A'], index = my_columns1), ignore_index = True)
/tmp/ipykernel_239/4081584381.py:20: FutureWarning: The frame.append method is deprecated and 

## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

The next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [5]:
simple_df.sort_values('One-Year Return', ascending = False, inplace = True)
simple_df = simple_df[:50]
simple_df.reset_index(inplace = True, drop = True)

simple_df

,Ticker,Price Today,Previous Price,One-Year Return,Number of Shares to Buy
0,META,314.60,88.9100,2.538410,N/A
1,NVDA,450.05,134.2100,2.353327,N/A
2,PHM,83.37,37.2200,1.239925,N/A
3,FICO,930.41,436.6800,1.130645,N/A
4,AVGO,882.68,444.1200,0.987481,N/A
5,ADBE,563.66,285.9300,0.971322,N/A
6,AMD,112.25,60.1100,0.867410,N/A
7,SNPS,487.94,279.2900,0.747073,N/A
8,CDNS,249.54,142.8600,0.746745,N/A
9,ANET,212.49,122.2500,0.738160,N/A


## Calculating the Number of Shares to Buy

In [6]:
def portfolio_input():

    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number!')
        print('Please try again: ')
        portfolio_size = input('Enter the size of your portfolio: ')

In [7]:
# portfolio_input()

portfolio_size = 10000000

position_size = float(portfolio_size) / len(simple_df.index)

In [8]:
for i in range(0, len(simple_df)):

    simple_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / simple_df.loc[i, 'Price Today'])

simple_df

,Ticker,Price Today,Previous Price,One-Year Return,Number of Shares to Buy
0,META,314.60,88.9100,2.538410,635
1,NVDA,450.05,134.2100,2.353327,444
2,PHM,83.37,37.2200,1.239925,2398
3,FICO,930.41,436.6800,1.130645,214
4,AVGO,882.68,444.1200,0.987481,226
5,ADBE,563.66,285.9300,0.971322,354
6,AMD,112.25,60.1100,0.867410,1781
7,SNPS,487.94,279.2900,0.747073,409
8,CDNS,249.54,142.8600,0.746745,801
9,ANET,212.49,122.2500,0.738160,941


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be caused by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame.

In [9]:
hqm_columns = [
    'Ticker',
    'Price Today',
    'One-Year Price',
    'One-Year Return',
    'One-Year Percentile',
    'Six-Month Price',
    'Six-Month Return',
    'Six-Month Percentile',
    'Three-Month Price',
    'Three-Month Return',
    'Three-Month Percentile',
    'One-Month Price',
    'One-Month Return',
    'One-Month Percentile',
    'HQM Score',
    'Number of Shares to Buy'
]

hqm_df = pd.DataFrame(columns = hqm_columns)

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in our modified S&P 500 list. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

As explained above, we will gather the required data through API calls to Polygon. This process takes quite a while (about 10 minutes), as five API calls are required for each stock in our modifed S&P 500 list.

In [10]:
cur_date = '2023-11-03'
oneyr_date = '2022-11-03'
sixmn_date = '2023-05-03'
threemn_date = '2023-08-03'
onemn_date = '2023-10-03'

for stk in sp500_stocks['Ticker'][:]:

    cur_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{cur_date}/{cur_date}?apiKey={POLYGON_API_KEY}'
    cur_data = requests.get(cur_api).json()
    cur_results0 = cur_data['results']
    cur_results1 = cur_results0[0]
    cur_price = cur_results1['c']

    oneyr_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{oneyr_date}/{oneyr_date}?apiKey={POLYGON_API_KEY}'
    oneyr_data = requests.get(oneyr_api).json()
    oneyr_results0 = oneyr_data['results']
    oneyr_results1 = oneyr_results0[0]
    oneyr_price = oneyr_results1['c']

    sixmn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{sixmn_date}/{sixmn_date}?apiKey={POLYGON_API_KEY}'
    sixmn_data = requests.get(sixmn_api).json()
    sixmn_results0 = sixmn_data['results']
    sixmn_results1 = sixmn_results0[0]
    sixmn_price = sixmn_results1['c']

    threemn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{threemn_date}/{threemn_date}?apiKey={POLYGON_API_KEY}'
    threemn_data = requests.get(threemn_api).json()
    threemn_results0 = threemn_data['results']
    threemn_results1 = threemn_results0[0]
    threemn_price = threemn_results1['c']

    onemn_api = f'https://api.polygon.io/v2/aggs/ticker/{stk}/range/1/day/{onemn_date}/{onemn_date}?apiKey={POLYGON_API_KEY}'
    onemn_data = requests.get(onemn_api).json()
    onemn_results0 = onemn_data['results']
    onemn_results1 = onemn_results0[0]
    onemn_price = onemn_results1['c']

    oneyr_return = (cur_price - oneyr_price) / oneyr_price
    sixmn_return = (cur_price - sixmn_price) / sixmn_price
    threemn_return = (cur_price - threemn_price) / threemn_price
    onemn_return = (cur_price - onemn_price) / onemn_price

    hqm_df = hqm_df.append(pd.Series([stk, 
                                      cur_price, 
                                      oneyr_price, 
                                      oneyr_return, 
                                      'N/A', 
                                      sixmn_price, 
                                      sixmn_return, 
                                      'N/A', 
                                      threemn_price, 
                                      threemn_return, 
                                      'N/A', 
                                      onemn_price, 
                                      onemn_return, 
                                      'N/A',
                                      'N/A',
                                      'N/A'], 
                                      index = hqm_columns), 
                                      ignore_index = True)

/tmp/ipykernel_239/1216760356.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(pd.Series([stk,
/tmp/ipykernel_239/1216760356.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(pd.Series([stk,
/tmp/ipykernel_239/1216760356.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(pd.Series([stk,
/tmp/ipykernel_239/1216760356.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_df = hqm_df.append(pd.Series([stk,
/tmp/ipykernel_239/1216760356.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co

Below, we calculate the percentile score for each stock for our four time periods.

In [11]:
time_periods = ['One-Year', 'Six-Month', 'Three-Month', 'One-Month']

for row in hqm_df.index:
    for period in time_periods:
        hqm_df.loc[row, f'{period} Percentile'] = score(hqm_df[f'{period} Return'], hqm_df.loc[row, f'{period} Return'])

## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
for row in hqm_df.index:
    momentum_perc = []
    
    for period in time_periods:
        momentum_perc.append(hqm_df.loc[row, f'{period} Percentile'])
    
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_perc)

## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_df.sort_values('HQM Score', ascending = False, inplace = True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(inplace = True, drop = True)

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to take in our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each of our 50 best momentum stocks.

In [14]:
portfolio_size = 10000000

In [15]:
position_size = float(portfolio_size) / len(hqm_df.index)

for i in hqm_df.index:
    hqm_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_df.loc[i, 'Price Today'])

We can find the number of shares to buy on the column to the far right of the DataFrame.

In [16]:
hqm_df

,Ticker,Price Today,One-Year Price,One-Year Return,One-Year Percentile,Six-Month Price,Six-Month Return,Six-Month Percentile,Three-Month Price,Three-Month Return,Three-Month Percentile,One-Month Price,One-Month Return,One-Month Percentile,HQM Score,Number of Shares to Buy
0,ANET,212.49,122.2500,0.738160,98.181818,132.73,0.600919,99.59596,177.760,0.195376,99.191919,184.89,0.149278,97.777778,98.686869,941
1,DLR,132.93,94.9500,0.400000,90.909091,94.79,0.402363,98.585859,120.850,0.099959,95.757576,115.15,0.154407,98.181818,95.858586,1504
2,ADBE,563.66,285.9300,0.971322,98.989899,345.25,0.632614,100.0,523.760,0.076180,91.515152,507.03,0.111690,89.494949,95.0,354
3,NOW,606.76,385.5600,0.573711,95.757576,437.08,0.388213,98.383838,560.400,0.082727,93.333333,541.24,0.121055,91.717172,94.79798,329
4,NWS,22.66,16.6800,0.358513,88.888889,17.19,0.318208,95.353535,20.140,0.125124,97.171717,20.14,0.125124,93.131313,93.636364,8826
5,FICO,930.41,436.6800,1.130645,99.393939,752.39,0.236606,89.89899,855.180,0.087970,93.939394,836.54,0.112212,89.69697,93.232323,214
6,NWSA,21.93,16.4600,0.332321,87.070707,17.03,0.287728,94.141414,19.710,0.112633,96.767677,19.43,0.128667,93.939394,92.979798,9119
7,AMZN,138.60,89.3000,0.552072,94.747475,103.65,0.337192,96.363636,128.910,0.075169,91.313131,124.72,0.111289,89.090909,92.878788,1443
8,PHM,83.37,37.2200,1.239925,99.59596,66.49,0.253873,92.121212,81.450,0.023573,80.40404,70.73,0.178708,98.989899,92.777778,2398
9,AIZ,163.50,123.4100,0.324852,86.666667,132.03,0.238355,90.10101,141.690,0.153928,97.979798,143.23,0.141521,96.161616,92.727273,1223
